In [ ]:
print(f"Initialising imports......")

import numpy as np
import pandas as pd
import os
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.utils import to_categorical

print(f"\nImports and Configurations Done Successfully!")

In [ ]:
IMG_SIZE = (128, 128)
BATCH_SIZE = 32
CLASSES = 43

In [ ]:
def create_dataframe(train_dir):
    data = []
    for class_id in range(CLASSES):
        class_path = os.path.join(train_dir, str(class_id))
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            data.append({'filepath': img_path, 'label': str(class_id)})
    df = pd.DataFrame(data)
    return df

train_dir = '/kaggle/input/gtsrb-german-traffic-sign/train'
test_csv = '/kaggle/input/gtsrb-german-traffic-sign/Test.csv'
test_img_dir = '/kaggle/input/gtsrb-german-traffic-sign'

In [ ]:
df = create_dataframe(train_dir)
train_df, val_df = train_test_split(
    df, test_size=0.15, stratify=df['label'], random_state=42
)
test_df = pd.read_csv(test_csv)
test_df['filepath'] = test_df['Path'].apply(lambda x: os.path.join(test_img_dir, x))
test_df['label'] = test_df['ClassId'].astype(str)

print(f"Dataframes Created")

In [ ]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

def make_generator(datagen, dataframe, shuffle=True):
    return datagen.flow_from_dataframe(
        dataframe,
        x_col='filepath',
        y_col='label',
        target_size=IMG_SIZE,
        class_mode='categorical',
        batch_size=BATCH_SIZE,
        shuffle=shuffle
    )

train_gen = make_generator(train_datagen, train_df)
val_gen = make_generator(val_datagen, val_df, shuffle=False)
test_gen = make_generator(test_datagen, test_df, shuffle=False)

In [ ]:
class_weights = class_weight.compute_class_weight(
    'balanced', classes=np.unique(train_df['label']), y=train_df['label']
)
class_weights = {i: class_weights[i] for i in range(CLASSES)}

In [ ]:
print(f"Building Model.....\n")
base_model = EfficientNetB0(
    include_top=False,
    weights='imagenet',
    input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)
)
base_model.trainable = False

inputs = Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
outputs = Dense(CLASSES, activation='softmax')(x)
model = Model(inputs, outputs)
print(f"\nModel Built Succesfully!")

In [ ]:
print(f"Initialising training....\n")
model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
callbacks = [
    EarlyStopping(patience=6, restore_best_weights=True, monitor='val_accuracy'),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6),
    ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_accuracy')
]

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=15,
    callbacks=callbacks,
    class_weight=class_weights
)

print(f"\nModel Trained!")

In [ ]:
base_model.trainable = True
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
history_ft = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=15,
    callbacks=callbacks,
    class_weight=class_weights
)

print(f"Model Tuned!")

In [ ]:
print(f"Evaluating the model....\n")
model.load_weights('best_model.h5')
test_loss, test_acc = model.evaluate(test_gen)
print(f"\nTest Accuracy: {test_acc * 100:.2f}%")

In [ ]:
print("\n=== DEFINING CLASS LABELS ===")
classes = {
            0: 'Speed limit (20km/h)',
            1: 'Speed limit (30km/h)',
            2: 'Speed limit (50km/h)',
            3: 'Speed limit (60km/h)',
            4: 'Speed limit (70km/h)',
            5: 'Speed limit (80km/h)',
            6: 'End of speed limit (80km/h)',
            7: 'Speed limit (100km/h)',
            8: 'Speed limit (120 km/h)',
            9: 'No passing',
           10: 'No passing veh over 3.5 tons',
           11: 'Right-of-way at intersection',
           12: 'Priority road',
           13: 'Yield',
           14: 'Stop',
           15: 'No vehicles',
           16: 'Veh > 3.5 tons prohibited',
           17: 'No entry',
           18: 'General caution',
           19: 'Dangerous curve left',
           20: 'Dangerous curve right',
           21: 'Double curve',
           22: 'Bumpy road',
           23: 'Slippery road',
           24: 'Road narrows on the right',
           25: 'Road work',
           26: 'Traffic signals',
           27: 'Pedestrians',
           28: 'Children crossing',
           29: 'Bicycles crossing',
           30: 'Beware of ice/snow',
           31: 'Wild animals crossing',
           32: 'End speed + passing limits',
           33: 'Turn right ahead',
           34: 'Turn left ahead',
           35: 'Ahead only',
           36: 'Go straight or right',
           37: 'Go straight or left',
           38: 'Keep right',
           39: 'Keep left',
           40: 'Roundabout mandatory',
           41: 'End of no passing',
           42: 'End no passing veh > 3.5 tons'
}
print(f"Defined {len(classes)} traffic sign classes")

In [ ]:
import matplotlib.pyplot as plt

def predict_traffic_sign(img_path):
    img = Image.open(img_path).convert('RGB').resize(IMG_SIZE)
    img = preprocess_input(np.array(img, dtype=np.float32))
    pred = model.predict(np.expand_dims(img, axis=0))
    class_id = np.argmax(pred)
    confidence = pred[0][class_id]
    class_name = classes[class_id] 
    return class_id, class_name, confidence, img

# Example prediction
img_path = '/kaggle/input/gtsrb-german-traffic-sign/Test/00001.png'
class_id, class_name, confidence, img_array = predict_traffic_sign(img_path)
print(f"Predicted: {class_name} (class {class_id}) with confidence: {confidence:.2%}")

plt.imshow(Image.open(img_path))
plt.title(f"Predicted: {class_name}\n({confidence:.2%})")
plt.axis('off')
plt.show()
